In [5]:
import os
import streamlit as st
import pickle
import time
import langchain
from langchain import OpenAI
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

In [3]:
!pip install langchain langchain-google-genai

   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------- ----------------- 0.8/1.4 MB 6.7 MB/s eta 0:00:01
   ------------------------------ --------- 1.0/1.4 MB 3.1 MB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 2.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/4.6 MB ? eta -:--:--
   ---- ----------------------------------- 0.5/4.6 MB 17.5 MB/s eta 0:00:01
   ------ --------------------------------- 0.8/4.6 MB 2.2 MB/s eta 0:00:02
   ----------- ---------------------------- 1.3/4.6 MB 2.2 MB/s eta 0:00:02
   --------------- ------------------------ 1.8/4.6 MB 2.0 MB/s eta 0:00:02
   ------------------ --------------------- 2.1/4.6 MB 2.0 MB/s eta 0:00:02
   ---------------------- ----------------- 2.6/4.6 MB 2.0 MB/s eta 0:00:01
   ------------------------ --------------- 2.9/4.6 MB 2.0 MB/s eta 0:00:01
   ----------------------------- ---------- 3.4/4.6 MB 1.9 MB/s eta 0:00:01
   ----------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.19.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<6.0.0dev,>=3.20.3, but you have protobuf 6.31.1 which is incompatible.


In [7]:
os.environ["GOOGLE_API_KEY"] = 'your_api_key'

In [69]:
#llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.7, ) 
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro-latest",   # <-- not gemini-1.5-flash
    temperature=0.3,
    max_tokens=500
)

In [71]:
loaders = UnstructuredURLLoader(urls=[
    "https://www.moneycontrol.com/news/business/markets/wall-street-rises-as-tesla-soars-on-ai-optimism-11351111.html",
    "https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html"
])
data = loaders.load() 
len(data)

2

In [72]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)

# As data is of type documents we can directly use split_documents over split_text in order to get the chunks.
docs = text_splitter.split_documents(data)

In [75]:
len(docs)

18

In [77]:
docs[0]

Document(metadata={'source': 'https://www.moneycontrol.com/news/business/markets/wall-street-rises-as-tesla-soars-on-ai-optimism-11351111.html'}, page_content='English\n\nHindi\n\nGujarati\n\nSpecials\n\nHello, Login\n\nHello, Login\n\nLog-inor Sign-Up\n\nMy Account\n\nMy Profile\n\nMy Portfolio\n\nMy Watchlist\n\nMy Alerts\n\nMy Messages\n\nPrice Alerts\n\nMy Profile\n\nMy PRO\n\nMy Portfolio\n\nMy Watchlist\n\nMy Alerts\n\nMy Messages\n\nPrice Alerts\n\nLogout\n\nLoans up to ₹50 LAKHS\n\nFixed Deposits\n\nCredit CardsLifetime Free\n\nCredit Score\n\nChat with Us\n\nDownload App\n\nFollow us on:\n\nNetwork 18\n\nGo Ad-Free\n\nMy Alerts\n\n>->MC_ENG_DESKTOP/MC_ENG_NEWS/MC_ENG_MARKETS_AS/MC_ENG_ROS_NWS_MKTS_AS_ATF_728\n\nMoneycontrol\n\nGo PRO NowPRO\n\nMoneycontrol PRO\n\nAdvertisement\n\nRemove Ad\n\nBusiness\n\nMarkets\n\nStocks\n\nEconomy\n\nCompanies\n\nTrends\n\nIPO\n\nOpinion\n\nEV Special\n\nHomeNewsBusinessMarketsWall Street rises as Tesla soars on AI optimism\n\nTrending Topic

In [79]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorindex = FAISS.from_documents(docs, embeddings)

In [80]:
file_path="vector_index.pkl"
with open(file_path, "wb") as f:
    pickle.dump(vectorindex, f)

In [81]:
if os.path.exists(file_path):
    with open(file_path, "rb") as f:
        vectorIndex = pickle.load(f)

In [82]:
from langchain.chains import RetrievalQA
chain = RetrievalQA.from_chain_type(llm=llm, chain_type="map_reduce", retriever=vectorindex.as_retriever())
chain

RetrievalQA(verbose=False, combine_documents_chain=MapReduceDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verbatim.\n______________________\n{context}'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='{question}'), additional_kwargs={})]), llm=ChatGoogleGenerativeAI(model='models/gemini-1.5-pro-latest', google_api_key=SecretStr('**********'), temperature=0.3, max_output_tokens=500, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000

In [83]:
query = "what is the price of Tiago iCNG?"
langchain.debug=True
chain({"query": query}, return_only_outputs=True)

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "what is the price of Tiago iCNG?"
}
[chain/start] [chain:RetrievalQA > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "The company also said it has also introduced the twin-cylinder technology on its Tiago and Tigor models.\n\nThe Tiago iCNG is priced between Rs 6.55 lakh and Rs 8.1 lakh, while the Tigor iCNG comes at a price range of Rs 7.8 lakh to Rs 8.95 lakh.\n\nTata Motors Passenger Vehicles Ltd Head-Marketing, Vinay Pant said these introductions put together will make the company's CNG line up \"appealing, holistic, and stronger than ever\".\n\nPTI\n\nfirst published: Aug 4, 2023 02:17 pm\n\nDiscover the latest Business News, Sensex, and Nifty updates. Obtain Personal Finance insights, tax queries, and expert opinions on Mon

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised NotFound: 404 models/gemini-1.5-pro-latest is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised NotFound: 404 models/gemini-1.5-pro-latest is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods..
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 8.0 seconds as it raised NotFound: 404 models/gemini-1.5-pro-latest is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods..
Retrying langchain_google_genai.chat_models._c

[llm/error] [chain:RetrievalQA > chain:MapReduceDocumentsChain > chain:LLMChain > llm:ChatGoogleGenerativeAI] [64.37s] LLM run errored with error:
"NotFound('models/gemini-1.5-pro-latest is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.')Traceback (most recent call last):\n\n\n  File \"C:\\Users\\user\\anaconda3\\Lib\\site-packages\\langchain_core\\language_models\\chat_models.py\", line 842, in generate\n    self._generate_with_cache(\n\n\n  File \"C:\\Users\\user\\anaconda3\\Lib\\site-packages\\langchain_core\\language_models\\chat_models.py\", line 1091, in _generate_with_cache\n    result = self._generate(\n             ^^^^^^^^^^^^^^^\n\n\n  File \"C:\\Users\\user\\anaconda3\\Lib\\site-packages\\langchain_google_genai\\chat_models.py\", line 1790, in _generate\n    response: GenerateContentResponse = _chat_with_retry(\n                                        ^^^^^^^^^^^^^^^

NotFound: 404 models/gemini-1.5-pro-latest is not found for API version v1beta, or is not supported for generateContent. Call ListModels to see the list of available models and their supported methods.